In [ ]:
import speech_recognition as sr
import cv2
import pytesseract
from gtts import gTTS
import os
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

def text_to_speech(text, language='en'):
    # Create a gTTS object
    tts = gTTS(text=text, lang=language)

    audio_file = 'speech.mp3'  # Choose your desired filename
    tts.save(audio_file)

    # Play the audio
    import io
    from IPython.display import Audio

# Save audio data to a BytesIO stream
    audio_data = io.BytesIO()
    tts.write_to_fp(audio_data)
    audio_data.seek(0)  # Rewind to the beginning of the stream

# Play the audio using IPython.display.Audio
    display(Audio(audio_data.read(), autoplay=True))

def perform_text_recognition(image_path):
    # Load the image using OpenCV
        img = cv2.imread(image_path)

    # Perform text recognition using Tesseract OCR
        text = pytesseract.image_to_string(img)

        return text

def auto_focus_capture(camera_index=0):
    # Open the camera
        cap = cv2.VideoCapture(camera_index)

    # Check if the camera opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera.")
            return

    # Create a window to display the camera feed
        cv2.namedWindow("Camera Feed", cv2.WINDOW_NORMAL)

        while True:
        # Capture frame-by-frame
            ret, frame = cap.read()

            if not ret:
                print("Error: Failed to capture frame.")
                break

        # Convert the frame to grayscale for image processing
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate the focus measure of the frame (e.g., using Laplacian)
            focus_measure = cv2.Laplacian(gray, cv2.CV_64F).var()

        # Set a threshold for focus measure (adjust as needed)
            focus_threshold = 100.0

        # If focus measure meets the threshold, capture the frame
            if focus_measure > focus_threshold:
                print("Image focused. Capturing...")
                cv2.imwrite("captured_image_text.jpg", frame)
                break

        # Display the camera feed
            cv2.imshow("Camera Feed", frame)

        # Check for key press to exit
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

    # Release the camera and close all OpenCV windows
        cap.release()
        cv2.destroyAllWindows()

# Function to activate object recognition
def activate_text_recognition():
    print("Activating text recognition...")
    pytesseract.pytesseract.tesseract_cmd = r'D:\\Tesseract-OCR\\tesseract.exe'
    config='digits'
# Call the auto_focus_capture function
    auto_focus_capture()
# Perform text recognition on the captured image
    captured_image_path = "captured_image_text.jpg"
    recognized_text = perform_text_recognition(captured_image_path)
    print("Recognized Text:")
    print(recognized_text)

# Converting the recognized text to speech
    text_to_speech(recognized_text)

# Function to activate text recognition
def activate_object_recognition():

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

    auto_focus_capture()
# Provide the file path of the image
    image_path = "captured_image_object.jpg"  # Replace this with the actual file path

# Open the image using PIL
    raw_image = Image.open(image_path).convert('RGB')

# unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt")

    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    print("Caption:")
    print(caption)

# Converting the recognized text to speech
    text_to_speech(caption)
    
# Initialize the recognizer
recognizer = sr.Recognizer()

# Function to listen for voice commands
def listen_for_commands():
    while True:  # Continuous listening loop
        with sr.Microphone() as source:
            print("Listening for commands...")
            audio = recognizer.listen(source)

        try:
            # Recognize speech using Google Speech Recognition
            command = recognizer.recognize_google(audio).lower()
            print("Command recognized:", command)

            # Check the recognized command and activate corresponding models
            if "what is in front of me" in command:
                activate_object_recognition()
            elif "read the text" in command:
                activate_text_recognition()
            elif "stop" in command:  # Add a stop command to exit the loop
                print("Stopped Listening!!")
                break  # Exit the loop
            else:
                print("Unknown command. Please try again.")

        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio.")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))

# Start listening for commands
listen_for_commands()

Listening for commands...
Command recognized: what is in front of me
Image focused. Capturing...


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Caption:
there is a man with glasses sitting in a room with a window


Listening for commands...
